In [1]:
from utils import *

In [14]:
def spectral_convolution(spectrum):
    convolution = []
    spectrum = sorted(spectrum)
    l_heavy = spectrum[1:]
    for i, m2 in enumerate(spectrum[:-1]):
        for m1 in l_heavy[i:]:
            diff = m1-m2
            #if 57 <= diff <= 200: 
            if diff != 0:
                convolution.append(diff)
    return sorted(convolution)

In [15]:
spectrum = [0, 137, 186, 323]
" ".join([str(i) for i in spectral_convolution(spectrum)])

'49 137 137 186 186 323'

In [13]:
137+186

323

In [16]:
with open("../data/dataset_104_4.txt") as fin:
    spectrum = [int(i.strip()) for i in fin.read().split(" ")]
    print(" ".join([str(i) for i in spectral_convolution(spectrum)]))

2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 14 14 22 22 22 22 22 22 22 22 22 22 22 22 23 23 23 23 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 30 30 32 32 32 32 32 32 32 32 32 32 32 32 32 33 33 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 35 35 35 35 35 35 35 35 36 36 36 36 36 36 36 36 37 37 42 42 44 44 44 44 44 44 45 45 45 45 45 45 45 45 46 46 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 56 56 56 56 56 56 56 56 57 57 58 58 58 58 58 58 58 58 59 59 66 66 66 66 66 66 66 66 66 66 67 67 68 68 68 68 68 68 68 68 68 68 68 68 69 69 69 69 69 69 69 69 69 69 69 69 69 69 69 69 69 69 70 70 71 71 71 71 71 71 71 71 71 71 71 71 71 71 71 71 71 71 79 79 79 79 79 79 79 79 79 79 79 79 79 79 79 79 79 79 80 81 81 81 81 81 81 81 81 81 81 81 81 83 83 90 90 90 90 90 90 90 90 90 90 90 90 92 92 92 92 92 92 100 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 101 102 102 102 102 10

In [79]:
def trim_leaderboard(leaderboard, N):
    scores = sorted(leaderboard, key = lambda x: x[1], reverse=True)
    trimmed_leaderboard = scores[:N]
    for peptide, score in scores[N:]:
        if score == trimmed_leaderboard[-1][1]:
            trimmed_leaderboard.append((peptide, score))
    return [l for l in trimmed_leaderboard]
        
def expand_candidates(candidates, spectrum, masses):
    new_candidates = list()
    for candidate, _ in candidates:
        for m in masses:
            new_candidate = candidate + [m]
            #some error here: need to set cyclic=True to get the correct answer, but we should compute linear scores
            new_candidates.append((new_candidate, peptide_score(new_candidate, spectrum))) 
    return new_candidates
    
def leaderboard_cyclopeptide_sequencing(spectrum, N, allowed_masses):
    spectrum = sorted(spectrum)
    parent_mass = spectrum[-1]
    spectrum_counts = defaultdict(int)
    for i in spectrum:
        spectrum_counts[i] += 1
    
    candidate_peptides = [([],0)]
    leader_peptide = ""
    leader_score = 0
    other_leaders = []
    while candidate_peptides:
        candidate_peptides = expand_candidates(candidate_peptides, spectrum, allowed_masses)
        remaining_candidates = []
        for candidate_peptide, candidate_score in candidate_peptides:
            peptide_mass = sum(candidate_peptide)
            if peptide_mass == parent_mass:
                cyclic_score = peptide_score(candidate_peptide, spectrum, cyclic=True)
                if cyclic_score > leader_score:
                    leader_score = cyclic_score
                    leader_peptide = candidate_peptide
                elif cyclic_score == leader_score:
                    other_leaders.append((candidate_peptide, cyclic_score))
            elif peptide_mass < parent_mass:    
                remaining_candidates.append((candidate_peptide, candidate_score))
        candidate_peptides = trim_leaderboard(remaining_candidates, N)
    
    leaderpeptide_string = "-".join((str(m) for m in leader_peptide))
    for i, j in other_leaders:
        print(i, j)
    return leaderpeptide_string

def get_restricted_alphabet(spectrum, M):
    massCount = defaultdict(int)
    convolved_masses = spectral_convolution(spectrum)

    for m in convolved_masses:
        if 57 <= m <= 200:
            massCount[m] += 1
            
    sortedMasses = sorted(massCount.items(), key = lambda x: x[1], reverse=True)
    alphabet = sortedMasses[:M]
    for mass, count in sortedMasses[M:]:
        if count == alphabet[-1][1]:
            alphabet.append((mass,count))
    
    return sorted([a for a, _ in alphabet])
    
def convolution_cyclopeptide_sequencing(spectrum, M, N):
    allowed_masses = get_restricted_alphabet(spectrum, M)
    return leaderboard_cyclopeptide_sequencing(spectrum, N, allowed_masses)

In [82]:
M=20
N=60
spectrum = [57, 57, 71, 99, 129, 137, 170, 186, 194, 208, 228, 265, 285, 299, 307, 323, 356, 364, 394, 422, 493]
convolution_cyclopeptide_sequencing(spectrum, M, N)

[129, 194, 170] 7
[137, 170, 186] 7
[137, 186, 170] 7
[170, 129, 194] 7
[170, 137, 186] 7
[170, 186, 137] 7
[170, 194, 129] 7
[186, 137, 170] 7
[186, 170, 137] 7
[194, 129, 170] 7
[194, 170, 129] 7
[57, 137, 170, 129] 12
[71, 99, 186, 137] 13
[71, 137, 186, 99] 13
[71, 194, 129, 99] 13
[99, 71, 137, 186] 13
[99, 71, 194, 129] 13
[99, 129, 194, 71] 13
[99, 186, 137, 71] 13
[129, 99, 71, 194] 13
[129, 194, 71, 99] 13
[137, 71, 99, 186] 13
[137, 186, 99, 71] 13
[186, 99, 71, 137] 13
[186, 137, 71, 99] 13
[194, 71, 99, 129] 13
[194, 129, 99, 71] 13
[57, 137, 71, 99, 129] 20
[71, 99, 129, 57, 137] 20
[71, 137, 57, 129, 99] 20
[99, 71, 137, 57, 129] 20
[99, 129, 57, 137, 71] 20
[129, 57, 137, 71, 99] 20
[137, 57, 129, 99, 71] 20
[129, 99, 71, 137, 57] 20
[137, 71, 99, 129, 57] 20
[57, 129, 99, 71, 66, 71] 20
[57, 137, 71, 99, 58, 71] 20
[71, 99, 129, 57, 79, 58] 20
[99, 71, 137, 57, 58, 71] 20
[99, 129, 57, 71, 66, 71] 20
[99, 129, 57, 79, 58, 71] 20
[99, 71, 58, 57, 137, 71] 20
[99, 71, 58,

'57-129-99-71-57-80'

In [76]:
peps = [[99,71,137,57,57,72],
        [71,99,129,57,80,57],
       [71,99,129,57,57,80],
       [99,71,137,57,72,57]]

spectrum = [0, 57, 57, 71, 99, 129, 137, 170, 186, 194, 208, 228, 265, 285, 299, 307, 323, 356, 364, 394, 422, 493]
for pep in peps:
    print(get_spectrum(pep, cyclic=False))
    print(peptide_score(pep, spectrum, cyclic=False))

[0, 57, 57, 71, 72, 99, 114, 129, 137, 170, 186, 194, 208, 251, 265, 307, 322, 323, 364, 394, 421, 493]
17
[0, 57, 57, 71, 80, 99, 129, 137, 137, 170, 186, 194, 228, 266, 285, 299, 323, 356, 365, 422, 436, 493]
17
[0, 57, 57, 71, 80, 99, 114, 129, 137, 170, 186, 194, 228, 243, 285, 299, 323, 342, 356, 413, 422, 493]
17
[0, 57, 57, 71, 72, 99, 129, 129, 137, 170, 186, 194, 208, 265, 266, 307, 323, 337, 364, 394, 436, 493]
17


In [49]:
with open("../data/dataset_104_7.txt") as fin:
    f = fin.read()
    M, N, spectrum, _ = f.split("\n")
    spectrum = [int(i.strip()) for i in spectrum.split(" ")]
    N = int(N.strip())
    M = int(M.strip())
convolution_cyclopeptide_sequencing(spectrum, M, N)

[186, 156, 147, 160, 147, 128, 200, 128, 114, 103] 92
[114, 103, 186, 156, 147, 160, 147, 128, 200, 128] 92
[200, 128, 147, 160, 147, 156, 186, 103, 114, 128] 92
[147, 160, 147, 156, 186, 103, 114, 128, 200, 128] 92
[156, 186, 103, 114, 128, 200, 128, 147, 160, 147] 92
[128, 200, 128, 114, 103, 186, 156, 147, 160, 147] 92
[200, 128, 114, 103, 186, 156, 147, 160, 147, 128] 92
[160, 147, 156, 186, 103, 114, 128, 200, 128, 147] 92
[160, 147, 128, 200, 128, 114, 103, 186, 156, 147] 92
[103, 186, 156, 147, 160, 147, 128, 200, 128, 114] 92
[128, 200, 128, 147, 160, 147, 156, 186, 103, 114] 92
[128, 147, 160, 147, 156, 186, 103, 114, 128, 200] 92
[128, 114, 103, 186, 156, 147, 160, 147, 128, 200] 92
[147, 156, 186, 103, 114, 128, 200, 128, 147, 160] 92
[147, 128, 200, 128, 114, 103, 186, 156, 147, 160] 92
[103, 114, 128, 200, 128, 147, 160, 147, 156, 186] 92
[156, 147, 160, 147, 128, 200, 128, 114, 103, 186] 92
[200, 128, 147, 103, 57, 147, 156, 186, 114, 128, 103] 92
[114, 128, 97, 103, 128,

'57-147-156-186-103-114-128-97-103-128-147-103'

In [83]:
#quiz

In [84]:
spectrum = [0, 86, 160, 234, 308, 320, 382]
spectral_convolution(spectrum)

[12,
 62,
 74,
 74,
 74,
 74,
 86,
 86,
 148,
 148,
 148,
 160,
 160,
 222,
 222,
 234,
 234,
 296,
 308,
 320,
 382]